# Traveling Salesman Optimization

### Objective Function

<center> $\min \sum_{j} \sum_{i}C_{ij} \times b_{ij}$</center>

**Note 1:** We set $C_{ii}$ to be a very large number $M$ relative to other distances so that the solver does not select the city where it currently is as the next city to travel to.

**Note 2:** Any set of $b_{ij}$'s containing a subtour (i.e. skipping other cities and coming back. For example, City 3 --> City 4 --> City 3) will not be allowed.

We make use of the following equation:

<center> $u_i-u_j+Nb_{ij}\le N-1$

for $i\ne j$

$i=2,3,...N$

$j=2,3,...N$

$u_j \ge 0$</center>

Where $N$ is the total number of cities (in this case, $5$).

An example (City 3 --> City 4 --> City 3):

<center> $u_3-u_4+5b_{34}\le (5-1)=4$</center>

<center> $u_4-u_3+5b_{43}\le (5-1)=4$</center>

<center> $5(b_{34}+b_{43})\le =8$</center>

If both $b_{34}$ and $b_{43}$ are $1$, the inequality is $False
$. Therefore, we cannot travel to a city where we have already been.

**Note 3:** The following constraints are required so that we leave a city only once and enter a city only once.

<center> $\sum_{i} b_{ij}=1\space\space\space\space\sum_{j} b_{ij}=1$</center>


In [1]:
! pip install -q pyomo
! pip install cplex -q
!apt-get install -y -qq coinor-cbc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd


# Load the Excel file with data and parameters we will need
xls = pd.ExcelFile('/content/drive/MyDrive/Colab Notebooks/Optimization with Python/Linear Programming/Traveling Salesman Problem/TSP_Data.xlsx')

# Read each sheet
distances = xls.parse('TSP', index_col = 'Unnamed: 0')
distances.head()

,City1,City2,City3,City4,City5
City1,1000000,132,217,164,58
City2,132,1000000,290,201,79
City3,217,290,1000000,113,303
City4,164,201,113,1000000,196
City5,58,79,303,196,1000000


In [4]:
city_distance_dict = {}

for i in range(len(distances)):
  for j in range(len(distances.columns)):
    city_distance_dict[(i,j)] = distances.iloc[i,j]

In [5]:
city_distance_dict

{(0, 0): np.int64(1000000),
 (0, 1): np.int64(132),
 (0, 2): np.int64(217),
 (0, 3): np.int64(164),
 (0, 4): np.int64(58),
 (1, 0): np.int64(132),
 (1, 1): np.int64(1000000),
 (1, 2): np.int64(290),
 (1, 3): np.int64(201),
 (1, 4): np.int64(79),
 (2, 0): np.int64(217),
 (2, 1): np.int64(290),
 (2, 2): np.int64(1000000),
 (2, 3): np.int64(113),
 (2, 4): np.int64(303),
 (3, 0): np.int64(164),
 (3, 1): np.int64(201),
 (3, 2): np.int64(113),
 (3, 3): np.int64(1000000),
 (3, 4): np.int64(196),
 (4, 0): np.int64(58),
 (4, 1): np.int64(79),
 (4, 2): np.int64(303),
 (4, 3): np.int64(196),
 (4, 4): np.int64(1000000)}

Set up the sets and parameters

In [6]:
from pyomo.environ import *

model = ConcreteModel()

model.i = Set(initialize=set(range(len(distances))))
model.j = Set(initialize=set(range(len(distances.columns))))
model.ii = Set(initialize=set(range(1,len(distances))), ordered=True)

i = model.i
j = model.j
ii = model.ii

model.C = Param(i, j, initialize=city_distance_dict)
C = model.C



Set up the decision variabless

In [7]:
model.b = Var(i, j, domain=Binary)
b = model.b

model.u = Var(i, domain=NonNegativeIntegers)  # to avoid subtour
u = model.u

Set up the objective function

In [8]:
def objective_function(model):
  return sum(model.C[i,j] * model.b[i,j] for i in model.i for j in model.j)

model.obj = Objective(expr=objective_function, sense=minimize)

Define the first two constraints:

In [9]:
model.C1 = Constraint(i, rule=lambda model, i: sum(model.b[i,j] for j in model.j) == 1)
model.C2 = Constraint(j, rule=lambda model, j: sum(model.b[i,j] for i in model.i) == 1)

In [10]:
def Constraint3(model,i,j):
  if i != j:
    return u[i] - u[j] + len(distances) * model.b[i,j] <= len(distances) - 1
  else:
    return u[i] - u[j] == 0
model.C3 = Constraint(ii,j, rule=Constraint3)

model.C4 = Constraint(i, rule=lambda model, i: model.b[i,i] == 0)

Invoke the Solver

In [11]:
solve = SolverFactory('cbc')
results = solve.solve(model)
print(results)


Problem: 
- Name: unknown
  Lower bound: 668.0
  Upper bound: 668.0
  Number of objectives: 1
  Number of constraints: 22
  Number of variables: 24
  Number of binary variables: 25
  Number of integer variables: 30
  Number of nonzeros: 20
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.04
  Wallclock time: 0.1
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 7
  Error rc: 0
  Time: 0.17798233032226562
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



Output solver solution results

In [12]:
for i in model.i:
  for j in model.j:
    if model.b[i,j]() == 1:
      print(f'City {i} to City {j}')

print(f'\nObjective Function (Total distance) = {model.obj()}')

City 0 to City 4
City 1 to City 3
City 2 to City 0
City 3 to City 2
City 4 to City 1

Objective Function (Total distance) = 668.0
